# Tutorial 02: Multiple wrappers
This tutorial introduces the use of multiple data sources, and shows how can one exchange information between them interchangeably. The code given here is based on this [1] example and builds upon on the gentle introdcution on the CUDS API [2].

## Background
One of the main strengths of CUDS objects is their ability to share information between different underlying data sources interchangeably. Using osp-core's innerworkings a data source can be represented as a CUDS object. A data source can be in turn a database, a simulation engine, or any other software package, which is able to either generate or store information.

We refer to a CUDS object, which represents an underlying data source as a **wrapper**, as it wraps around the data source. Wrappers use the CUDS API with the addition to some wrapper-specific methods, which will be discussed later on in this tutorial.

For a wrapper to be initialized, one needs some context for the underlying data source (e.g. location, credentials, etc.) for this we introduce an object called **session**. Conceptually a session can be thought as an interoperability level, or in simple terms it handles the transition from the user-friendly CUDS API to the more task-specific syntax data sources tend to have.

## Let's get hands on
We start by importing the CUDS classes needed for this example.

In [ ]:
from cuds.classes import (
    City, Citizen, Neighbourhood, Street,
    HasPart, HasInhabitant, IsPartOf,
    CUBA, CityWrapper, CitySimWrapper, Person
)

Next we import a session object, which will provide the context to a simple simulation engine we developed for demonstrational purposes.

In [ ]:
from cuds.testing.test_sim_wrapper_city import DummySimSession

The `pretty_print` function is part of our utilities module and is a convinient way to output the tree-like structure of a CUDS object.

In [ ]:
from cuds.utils import pretty_print

The `getpass` function is used to retrieve an input from an user. It prints a prompt, then reads input from the user until they press return.

In [ ]:
from getpass import getpass

The next statement imports the second data source session we will use in this example, namely a database, or more precisely the ORM toolkit SQLAlchemy [4] which we will use in turn to connect to a PostgreSQL database.
One throws an error, if it cannot find the `SQLAlchemyWrapperSession` as it needs to be installed from a separate repository. Please refer here [5] for installation instructions.

In [ ]:
try:
    from cudsqlalchemy.sqlalchemy_wrapper_session import \
        SqlAlchemyWrapperSession
except ImportError as e:
    raise ImportError("For this example, the SQLAlchemy "
                      "wrapper for SimPhoNy is required!") from e

The following lines prompts the user to enter the information needed to create a connection to a running instance of a PostgreSQL database, where the data of our demonstrational simulation will be stored. Please make sure to point to an existing and running instance of PostgreSQL. To install PostgreSQL on your machine, please refer to their documentation [6].

The information is stored in a string `postgres_url`, which will later be passed to the `SQLAlchemyWrapperSession` object to initiate a connection with the data base.

In [ ]:
print("Input data to connect to Postgres table!")
user = input("User: ")
pwd = getpass("Password: ")
db_name = input("Database name: ")
host = input("Host: ")
port = int(input("Port [5432]: ") or 5432)
postgres_url = 'postgres://%s:%s@%s:%s/%s' % (user, pwd, host, port, db_name)

In the next lines we create our small ontology-loving **EMMO town** [7] example. Please pay a closer attention to the fourth line ([this](https://gitlab.cc-asp.fraunhofer.de/simphony/osp-core/blob/master/examples/multiple_wrappers_example.py#L31) line in the `multiple_wrappers_example.py` file) as there you can see the power of the `add` method and how with one statement one can add multiple `Citizen` CUDS objects simultaneously.

In [ ]:
emmo_town = City('EMMO town')

emmo_town.add(Citizen(name='Emanuele Ghedini'), rel=HasInhabitant)
emmo_town.add(Citizen(name='Adham Hashibon'), rel=HasInhabitant)
emmo_town.add(Citizen(name='Jesper Friis'), Citizen(name='Gerhard Goldbeck'),
              Citizen(name='Georg Schmitz'), Citizen(name='Anne de Baas'),
              rel=HasInhabitant)

emmo_town.add(Neighbourhood("Ontology"))
emmo_town.add(Neighbourhood("User cases"))

Next we grow the `Ontology` neighbourhood by adding some streets to it: namely *relationships* and *entities* (puns are intended).

In [ ]:
ontology_uid = None
for neighbourhood in emmo_town.get(cuba_key=CUBA.NEIGHBOURHOOD):
    if neighbourhood.name == "Ontology":
        ontology_uid = neighbourhood.uid
        neighbourhood.add(Street("Relationships"), rel=HasPart)
        neighbourhood.add(Street("Entities"), rel=HasPart)

The osp-core support multiple types of relationships and the `inverse` relationship is one of them. In our example case we retrieve from the "Ontology" neighbourhood `onto`, which town it belongs to.

In [ ]:
onto = emmo_town.get(ontology_uid)
print(onto.get(rel=IsPartOf)[0].name + ' is my city!')

Let's now store our small city persistently in the PostgreSQL database. Working with session objects is similar to the way one is used to work with files in Python. And when you ponder a bit about it, it makes kind of sense, as what one intends to do is to store some information to a data storage.

Using Python's `with` statement the connection to the database will be maintained only within the scope of the `with` statement. After exciting its scope the connection will be closed automatically. We advise the use of the `with` statement as it automatically manages opening and closing of database connections.

First we open a connection to a PostgreSQL database through our `SqlAlchemyWrapperSession` and assign it to the session variable. Then we assign to the `wrapper` variable a `CityWrapper` and pass it the needed context with the `session=session`. From there on, we treat the `wrapper` variable as a normal CUDS object with the only difference, that its internal state is managed by the `SqlAlchemyWrapperSession` behind the scenes. On the last line, we see the benefits of that by simply executing the `commit` command onto the `session` object. This will trigger an array of events, with the end result being that our **EMMO town** will be stored in the database.

In [ ]:
with SqlAlchemyWrapperSession(postgres_url) as session:
    wrapper = CityWrapper(session=session)
    wrapper.add(emmo_town)
    session.commit()

Next we show how one can use multiple data source wrappers simultaneously. First we open a connection to the PostgreSQL database through `SqlAlchemyWrapperSession` as shown above and assign it to the `db_session` variable. Then we retrieve the information we have previously stored to it using CUDS `get` method and use the `pretty_print` function to output the information.

Then we open a connection to our demonstrational simulation engine through `DummySimSession` and assign it to the `sim_session` variable. As you can see opening the second simulation session is within the scope of the `SqlAlchemyWrapperSession`. 

In the scope of `DummySimSession`, we initialize a `CitySimWrapper`, pass it the context from the `sim_session` and assign it to the `sim_wrapper` variable. The `CitySimWrapper` consumes a city and a person. The magic happens in the following `run` method, which recognise that **Peter** is a person and it then transforms him into a citizen of the **EMMO town**. This new information is then stored in the `sim_emmo_town` automatically within the `run` method. We then output the information about Peter, who is now a citizen of **EMMO town**.

Finally we `update` our now outdated **EMMO town** in the database by using the `update` command. It checks for any inconsistencies between the **EMMO town** stored in the database, `db_emmo_town`, and the modified by our simulation engine town, `sim_emmo_town`. In our case it will find its new citizen **Peter** and it will add it to the **EMMO town** in the database instance of the town, `db_emmo_town`. This change is then in turn made persistent by calling the `commit` method, which will actually store the information the database.

In [ ]:
with SqlAlchemyWrapperSession(postgres_url) as db_session:
    db_wrapper = CityWrapper(session=db_session)
    db_emmo_town = db_wrapper.get(emmo_town.uid)
    print("The database contains the following information about the city:")
    pretty_print(db_emmo_town)

    # Working with a Simulation wrapper
    with DummySimSession() as sim_session:
        sim_wrapper = CitySimWrapper(num_steps=1,
                                     session=sim_session)
        new_inhabitant = Person(age=31, name="Peter")
        sim_emmo_town, _ = sim_wrapper.add(db_emmo_town, new_inhabitant)
        sim_session.run()
        print("The city has a new inhabitant:")
        pretty_print(sim_emmo_town.get(new_inhabitant.uid))

    # update database
    db_wrapper.update(sim_emmo_town)
    db_session.commit()


Finally to ensure our data base has successfully interpreted our addition to **Emmo town**, we check its contents and print them using `pretty_print`.

In [ ]:
with SqlAlchemyWrapperSession(postgres_url) as db_session:
    db_wrapper = CityWrapper(session=db_session)
    db_emmo_town = db_wrapper.get(emmo_town.uid)
    print("The database contains the following information about the city:")
    pretty_print(db_emmo_town)

## References
* [1] https://gitlab.cc-asp.fraunhofer.de/simphony/osp-core/blob/master/examples/multiple_wrappers_example.py
* [2] https://gitlab.cc-asp.fraunhofer.de/simphony/getting-started/blob/master/Tutorials/cuds-api.ipynb
* [3] https://en.wikipedia.org/wiki/Create,_read,_update_and_delete
* [4] https://www.sqlalchemy.org/
* [5] https://gitlab.cc-asp.fraunhofer.de/simphony/wrappers/sqlalchemy-wrapper
* [6] https://www.postgresql.org/download/
* [7] https://emmc.info/emmo-info/
